# Convert NeuCosmA files
The file format is as following (by column):

1. parent id
2. daughter id
3. systematic flag (currently ignored)
4. log10(E [GeV]) (E or eps_r or y depending on corresp. column)
5. g_ij(y) [mubarn = 10^-30 cm^2]
6. M_ij(eps_r)
7. f_i(y) [mubarn = 10^-30 cm^2]
8. sigma_i(eps_r) [mubarn = 10^-30 cm^2]

In [1]:
import numpy as np
import pandas as pd

In [2]:
data_table = pd.read_csv('./prince_data_utils/resources/photo-nuclear/160513_XDIS_PSB-SS_syst.dat',delim_whitespace=True,
            names=['mother','daughter','sys_flag','log10(E)','g(y)','M(eps)','f_i(y)','sigma_i'])
data_table = data_table.set_index(['mother','daughter'])
data_table['dsigma_ij'] = data_table['sigma_i'] * data_table['M(eps)'] 
data_table

sys_flag      log10(E)     g(y)  M(eps)    f_i(y)  sigma_i  \
mother daughter                                                               
201    100              0 -3.500000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.450000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.400000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.350000e+00   0.0000     0.0    0.0000      0.0   
       100              0 -3.300000e+00   0.0000     0.0    0.0000      0.0   
...                   ...           ...      ...     ...       ...      ...   
5626   5525             1 -2.000000e-01  18.0244     0.0  140.5530      0.0   
       5525             1 -1.500000e-01  16.0738     0.0  100.1920      0.0   
       5525             1 -1.000000e-01  15.9104     0.0   92.9049      0.0   
       5525             1 -5.000000e-02  12.2481     0.0   46.0163      0.0   
       5525             1  1.928650e-16  12.0852     0.0   34.6745      0.0   

                 dsigma_ij  
mother daughter             
201    100             0.0  
       100             0.0  
       100             0.0  
       100             0.0  
       100             0.0  
...                    ...  
5626   5525            0.0  
       5525            0.0  
       5525            0.0  
       5525            0.0  
       5525            0.0  

[48493 rows x 7 columns]

# Consistency checks

Before converting, we need to check two things:

- The energy grid is the same for all channels (not guaranteed for NEUCOSMA, but requried for PriNCe)
- The nonel cross section should be the same for all mother (It is stored redundantly)

In [3]:
last_idx = None
last_egrid = None
last_csec  = None
for index, data in data_table.groupby(level=[0,1]):
    if last_idx is None:
        last_idx = index
        last_egrid = data['log10(E)']
        last_csec = data['sigma_i']

    assert np.all(last_egrid.values == data['log10(E)'].values)
    if index[0] == last_idx[0]:
        assert np.all(last_csec.values == data['sigma_i'].values)
 
    last_idx = index
    last_egrid = data['log10(E)']
    last_csec = data['sigma_i']   
        
    #print(data['sigma_i']) 

# Now write this to datafiles

In [4]:
egrid_peanut = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_egrid.dat.bz2')
nonel_peanut = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_nonel.dat.bz2')
incl_peanut  = np.loadtxt('./prince_data_utils/resources/photo-nuclear/PEANUT_IAS_incl_i_j.dat.bz2')

print(egrid_peanut.shape)
print(nonel_peanut.shape)
print(incl_peanut.shape)

(148,)
(405, 149)
(49386, 150)


In [5]:
egrid = 10**data_table[['log10(E)']].drop_duplicates()
egrid = egrid.values
egrid = egrid.flatten()

print(egrid.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_egrid.dat.bz2',egrid)

(71,)


In [6]:
table = data_table.reset_index()
table = table[['mother','log10(E)','sigma_i']].drop_duplicates()
table = table.drop('log10(E)',axis='columns')

table = table.set_index('mother')
print(table.shape)

#array = [np.array(101)[None],np.zeros_like(egrid),]
array = []
array.append(np.concatenate([np.array(101)[None],np.zeros_like(egrid)]))
for index, data in table.groupby(level=[0]):
    data = data.T.values.flatten() * 1e-3 # mubarn to mbarn
    array.append(np.concatenate([np.array(index)[None],data]))   
nonel = np.array(array)
print(nonel.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_nonel.dat.bz2',nonel)

(3621, 1)
(52, 72)


In [7]:
table = data_table.reset_index()
table = table[['mother','daughter','log10(E)','dsigma_ij']].drop_duplicates()
table = table.drop('log10(E)',axis='columns')

table = table.set_index(['mother','daughter'])
print(table.shape)

array = []
for index, data in table.groupby(level=[0,1]):
    data = data.T.values.flatten() * 1e-3 # mubarn to mbarn
    array.append(np.concatenate([np.array(index),data]))
incl = np.array(array)
print(incl.shape)
np.savetxt('./prince_data_utils/resources/photo-nuclear/PSB_incl_i_j.dat.bz2',incl)

(48493, 1)
(683, 73)
